In [12]:
pip install mlxtend

In [13]:
import pandas as pd
from itertools import combinations
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [14]:
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
from mlxtend.frequent_patterns import association_rules

In [15]:
# Function to generate frequent itemsets using brute-force method
def generate_frequent_itemsets_bruteforce(transactions, min_support):
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item in item_counts:
                item_counts[item] += 1
            else:
                item_counts[item] = 1

    frequent_itemsets = []
    for item, count in item_counts.items():
        support = count / len(transactions)
        if support >= min_support:
            frequent_itemsets.append({item})

    k = 2
    while True:
        candidate_itemsets = set(combinations(item_counts.keys(), k))
        frequent_itemsets_k = []
        for candidate in candidate_itemsets:
            support = sum(1 for transaction in transactions if set(candidate).issubset(set(transaction))) / len(transactions)
            if support >= min_support:
                frequent_itemsets_k.append(set(candidate))
        if not frequent_itemsets_k:
            break
        frequent_itemsets.extend(frequent_itemsets_k)
        k += 1

    return frequent_itemsets

# Function to convert transactions into a boolean DataFrame
def create_boolean_df(transactions):
    unique_items = sorted(set(item for sublist in transactions for item in sublist))
    boolean_df = pd.DataFrame(0, columns=unique_items, index=range(len(transactions)))
    for i, transaction in enumerate(transactions):
        boolean_df.loc[i, transaction] = 1
    return boolean_df

# Function to generate association rules using Apriori algorithm
def generate_association_rules_apriori(transactions, min_support, min_confidence):
    df = create_boolean_df(transactions)
    frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    return rules

# Function to generate association rules using FP-Growth algorithm
def generate_association_rules_fpgrowth(transactions, min_support, min_confidence):
    df = create_boolean_df(transactions)
    frequent_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)
    return rules

# Set minimum support and confidence
min_support = 0.2
min_confidence = 0.7

In [16]:
# Load transactions from CSV files for each dataset
amazon_transactions = pd.read_csv("amazon_transactions.csv")['Transaction'].apply(lambda x: x.split(", ")).tolist()
best_buy_transactions = pd.read_csv("best_buy_transactions.csv")['Transaction'].apply(lambda x: x.split(", ")).tolist()
k_mart_transactions = pd.read_csv("k_mart_transactions.csv")['Transaction'].apply(lambda x: x.split(", ")).tolist()
nike_transactions = pd.read_csv("nike_transactions.csv")['Transaction'].apply(lambda x: x.split(", ")).tolist()
flipkart_transactions = pd.read_csv("flipkart_transactions.csv")['Transaction'].apply(lambda x: x.split(", ")).tolist()

# Timing performance and generating association rules for each dataset
datasets = {
    "Amazon": amazon_transactions,
    "Best Buy": best_buy_transactions,
    "K-mart": k_mart_transactions,
    "Nike": nike_transactions,
    "Flipkart": flipkart_transactions
}

for dataset_name, dataset_transactions in datasets.items():
    print(f"\n{dataset_name} Timing Performance:")

    # Brute-force method
    start_time = time.time()
    brute_force_frequent_itemsets = generate_frequent_itemsets_bruteforce(dataset_transactions, min_support)
    brute_force_execution_time = time.time() - start_time
    print("Brute-force Execution Time:", brute_force_execution_time, "seconds")

    # Apriori algorithm
    start_time = time.time()
    apriori_association_rules = generate_association_rules_apriori(dataset_transactions, min_support, min_confidence)
    apriori_execution_time = time.time() - start_time
    print("Apriori Execution Time:", apriori_execution_time, "seconds")

    # FP-Growth algorithm
    start_time = time.time()
    fpgrowth_association_rules = generate_association_rules_fpgrowth(dataset_transactions, min_support, min_confidence)
    fpgrowth_execution_time = time.time() - start_time
    print("FP-Growth Execution Time:", fpgrowth_execution_time, "seconds")

    # Print association rules for each algorithm
    print(f"\n{dataset_name} Association Rules:")
    print("Brute-force:")
    print(brute_force_frequent_itemsets)
    print("Apriori:")
    print(apriori_association_rules)
    print("FP-Growth:")
    print(fpgrowth_association_rules)


Amazon Timing Performance:
Brute-force Execution Time: 0.172410249710083 seconds
Apriori Execution Time: 0.026085376739501953 seconds
FP-Growth Execution Time: 0.026730060577392578 seconds

Amazon Association Rules:
Brute-force:
[{"['A Beginner’s Guide'"}, {"'Java: The Complete Reference'"}, {"'Java For Dummies'"}, {"'Android Programming: The Big Nerd Ranch']"}, {"'Android Programming: The Big Nerd Ranch'"}, {"'Head First Java 2nd Edition']"}, {"'Android Programming: The Big Nerd Ranch']", "'Java: The Complete Reference'"}, {"['A Beginner’s Guide'", "'Java: The Complete Reference'"}, {"['A Beginner’s Guide'", "'Java For Dummies'"}, {"'Java: The Complete Reference'", "'Java For Dummies'"}, {"'Android Programming: The Big Nerd Ranch']", "'Java For Dummies'"}, {"'Android Programming: The Big Nerd Ranch']", "['A Beginner’s Guide'"}, {"'Android Programming: The Big Nerd Ranch']", "['A Beginner’s Guide'", "'Java For Dummies'"}, {"'Android Programming: The Big Nerd Ranch']", "['A Beginner’s 

In [17]:
# COMPARISON:

# Amazon Timing Performance:
# Brute-force Execution Time: 0.2229 seconds
# Apriori Execution Time: 0.0220 seconds
# FP-Growth Execution Time: 0.0220 seconds

# Best Buy Timing Performance:
# Brute-force Execution Time: 0.1811 seconds
# Apriori Execution Time: 0.0504 seconds
# FP-Growth Execution Time: 0.0320 seconds

# K-mart Timing Performance:
# Brute-force Execution Time: 0.1694 seconds
# Apriori Execution Time: 0.0351 seconds
# FP-Growth Execution Time: 0.0250 seconds

# Nike Timing Performance:
# Brute-force Execution Time: 2.1365 seconds
# Apriori Execution Time: 0.0470 seconds
# FP-Growth Execution Time: 0.0409 seconds

# Flipkart Timing Performance:
# Brute-force Execution Time: 0.0100 seconds
# Apriori Execution Time: 0.0180 seconds
# FP-Growth Execution Time: 0.0160 seconds

# Regarding whether the three algorithms produce the same results, it depends on the dataset and the support and confidence thresholds used.
# While the algorithms aim to find frequent itemsets and association rules, the specific implementation details may lead to slight variations in the results.
# It's essential to compare the association rules generated by each algorithm to determine if they are consistent across the board.

# In terms of speed, the FP-Growth algorithm tends to be faster compared to both the Brute-force and Apriori algorithms for most datasets.
# However, the actual performance may vary depending on the dataset size, itemset complexity, and the specific implementation of each algorithm.
